In [1]:
import tensorflow as tf
import numpy as np
import os
import sys
import skimage
import pylab as plt
from importlib import reload

sys.path.insert(0, '../src')
import deepleeo.dataset.data_augment as dtaug
import deepleeo.dataset.utils as dsutils
from deepleeo.networks import fcn

reload(dtaug)
reload(dsutils)
reload(fcn)

/home/raian/anaconda3/envs/tensorflow/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


<module 'deepleeo.networks.fcn' from '../src/deepleeo/networks/fcn.py'>

# Load input Dataset

In [2]:
DATA_DIR = os.path.join(os.path.abspath(os.path.dirname("__file__")), '../', 'data_real', 'generated')
DATASET_FILE = os.path.join(DATA_DIR, 'samples_dataset.npz')

In [3]:
dataset = np.load(DATASET_FILE)

print("Data Loaded:")
print("  -> Images: ", len(dataset["images"]))
print("  -> Labels: ", len(dataset["labels"]))
print("  -> Classes: ", len(dataset["classes"]))

print("Images shape: ", dataset["images"][0].shape)
print("Labels shape: ", dataset["labels"][0].shape)

Data Loaded:
  -> Images:  5000
  -> Labels:  5000
  -> Classes:  4
Images shape:  (128, 128, 3)
Labels shape:  (128, 128, 1)


In [4]:
#plt.figure(figsize=(4,4))
#img_plt = skimage.img_as_float(dataset["images"][0][:,:,[5, 4, 3]]).astype(np.float32)
#img_plt = dataset["images"][0][:,:,[5, 4, 3]]
#plt.imshow(img_plt)
#plt.axis('off')

# Perform Data Augmentation

In [5]:
angles = [90, 180, 270]
rotated_imgs = dtaug.rotate_images(dataset["images"], angles)
flipped_imgs = dtaug.flip_images(dataset["images"])

new_dataset = {}
new_dataset["images"] = np.concatenate((dataset["images"], rotated_imgs))
new_dataset["images"] = np.concatenate((new_dataset["images"], flipped_imgs))

rotated_lbls = dtaug.rotate_images(dataset["labels"], angles)
flipped_lbls = dtaug.flip_images(dataset["labels"])

new_dataset["labels"] = np.concatenate((dataset["labels"], rotated_lbls))
new_dataset["labels"] = np.concatenate((new_dataset["labels"], flipped_lbls))

new_dataset["classes"] = dataset["classes"]

print("Data Augmentation Applied:")
print("  -> Images: ", new_dataset["images"].shape)
print("  -> Labels: ", new_dataset["labels"].shape)

Data Augmentation Applied:
  -> Images:  (35000, 128, 128, 3)
  -> Labels:  (35000, 128, 128, 1)


# Split dataset between train, test and validation data

In [6]:
train_images, test_images, valid_images, train_labels, test_labels, valid_labels = dsutils.split_dataset(new_dataset)

print("Splitted dataset:")
print("  -> Train images: ", train_images.shape)
print("  -> Test images: ", test_images.shape)
print("  -> Validation images: ", valid_images.shape)
print("  -> Train Labels: ", train_labels.shape)
print("  -> Test Labels: ", test_labels.shape)
print("  -> Validation Labels: ", valid_labels.shape)

Splitted dataset:
  -> Train images:  (24500, 128, 128, 3)
  -> Test images:  (10500, 128, 128, 3)
  -> Validation images:  (0, 128, 128, 3)
  -> Train Labels:  (24500, 128, 128, 1)
  -> Test Labels:  (10500, 128, 128, 1)
  -> Validation Labels:  (0, 128, 128, 1)


# Train the Network

In [44]:
reload(fcn)
hyper_params = {
    "epochs": 10,
    "batch_size": 256,
    "learning_rate": 0.0001,
    "class_names": new_dataset["classes"]
}

fcn.fcn_train(train_images, test_images, train_labels, test_labels, hyper_params, DATA_DIR)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_keep_checkpoint_every_n_hours': 10000, '_model_dir': '/home/raian/doutorado/DeepLeEO/notebooks/../data_real/generated', '_task_id': 0, '_session_config': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f7b616b1208>, '_is_chief': True, '_num_worker_replicas': 1, '_tf_random_seed': None, '_save_checkpoints_steps': None, '_log_step_count_steps': 100, '_train_distribute': None, '_evaluation_master': '', '_master': '', '_num_ps_replicas': 0, '_device_fn': None, '_save_summary_steps': 100, '_service': None, '_global_id_in_cluster': 0, '_task_type': 'worker', '_keep_checkpoint_max': 5}
INFO:tensorflow:Calling model_fn.


ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'Layer_1_1/conv2d/kernel:0' shape=(3, 3, 3, 64) dtype=float32_ref>", "<tf.Variable 'Layer_1_1/conv2d/bias:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'Layer_1_1/batch_normalization/gamma:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'Layer_1_1/batch_normalization/beta:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'Layer_1_2/conv2d/kernel:0' shape=(3, 3, 64, 64) dtype=float32_ref>", "<tf.Variable 'Layer_1_2/conv2d/bias:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'Layer_1_2/batch_normalization/gamma:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'Layer_1_2/batch_normalization/beta:0' shape=(64,) dtype=float32_ref>", "<tf.Variable 'Layer_2_1/conv2d/kernel:0' shape=(3, 3, 64, 128) dtype=float32_ref>", "<tf.Variable 'Layer_2_1/conv2d/bias:0' shape=(128,) dtype=float32_ref>", "<tf.Variable 'Layer_2_1/batch_normalization/gamma:0' s